<a href="https://colab.research.google.com/github/cheikhsaliou01/-3/blob/main/Questionnary_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install deep_translator
from deep_translator import GoogleTranslator
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00


In [15]:
!pip install pymorphy3
!pip install nltk
import pymorphy3
import re
from nltk import word_tokenize, pos_tag

In [23]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_rus')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_rus to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_rus.zip.


True

In [5]:
def to_norm(w): # перевод слова в нормальную форму
    morph = pymorphy3.MorphAnalyzer()
    p = morph.parse(w)[0]
    return p.normal_form

def months(s): # перевод даты в порядковый номер месяца
    l = s.split('-')
    m, y = int(l[1]), int(l[0])
    mm, my = 10, 2021
    return (y-my)*12 + m-mm

def preoc_class(x): # перевод первичного класса в порядковый номер
    try:
        l = x.split('-')
        return int(l[0])
    except:
        return 0
def preoc2_class(x): # перевод вторичного класса в порядковый номер
    try:
        l = x.split('-')
        return l[0]
    except:
        return None

def pairs(f_apeal): # построение существительно-глаголных пар
    try:
        text = f_apeal[:5000]
    except:
        return None
    apeal = GoogleTranslator(source='fr', target='ru').translate(text=f_apeal[:5000])
    words_list = word_tokenize(apeal)
    wl = [re.sub('[^а-яё]','',wli.lower()) for wli in words_list if len(wli)>1]
    wl = list(filter(('').__ne__, wl))
    parts = pos_tag(wl, lang = 'rus')
    f1,f2 = 0, 0
    P_list = []
    for part in parts:
        if len(part[0]) == 1:
            continue
        tp = to_norm(part[0])
        if part[1][0] == 'S':
            first_part = tp
            f1+=1
        if part[1] == 'V':
            second_part = tp
            f2+=1
        if f1 > 0 and f2 > 0:
            if (first_part, None) in P_list:
                P_list.remove((first_part, None))
            elif (None, second_part) in P_list:
                P_list.remove((None, second_part))
            P_list.append((first_part,second_part))
            f1, f2 = 0, 0
        elif f1 > 0 and (first_part not in [p[0] for p in P_list]):
            P_list.append((first_part, None))
        elif f2 > 0 and (second_part not in [p[1] for p in P_list]):
            P_list.append((None, second_part))
    return P_list

def involve(row_pairs, k, train_dict): # учет нового набора пар для класса k при обучении
    if k not in train_dict:
        train_dict[k] = {}
    if not row_pairs:
        return train_dict
    for noun, verb in row_pairs:
        if noun:
            if noun not in train_dict[k]:
                train_dict[k][noun] = {}
            if verb:
                if verb not in train_dict[k][noun]:
                    train_dict[k][noun][verb] = 3
                else:
                    train_dict[k][noun][verb] += 3
            else:
                if '-' not in train_dict[k][noun]:
                    train_dict[k][noun]['-'] = 2
                else:
                    train_dict[k][noun]['-'] += 2
        else:
            for kn in train_dict[k]:
                if verb in train_dict[k][kn]:
                    train_dict[k][kn][verb] += 1
    return train_dict

In [6]:
def classify(text, train_dict, ncats): # классификатор на первичные или вторичные классы (определяется списком ncats)
    text_pairs = pairs(text)
    if not text_pairs:
        return 0
    app_dict = {}
    for noun, verb in text_pairs:
        for cat in ncats:
            if cat not in train_dict:
                continue
            if cat not in app_dict:
                app_dict[cat] = 0
            total = sum(sum(dv.values()) for dv in train_dict[cat].values())
            if noun not in train_dict[cat]:
                continue
            if verb not in train_dict[cat][noun]:
                if '-' in train_dict[cat][noun]:
                    verb = '-'
                else:
                    continue
            app_dict[cat] += train_dict[cat][noun][verb]/total
    return max(app_dict, key=app_dict.get)

In [9]:
# чтение данных
data = pd.read_excel('data_vigie_finalelarge.xlsx', sheet_name='Worksheet', usecols='B:H')
data.head()

,MRC_ok,Type d'Éclaireurs,Date_vigie_ok,Preoccup,Preoccup_ok,Categorie_preoccup_nv1_ok,Categorie_preoccup_nv2
0,MRC de la Nouvelle-Beauce,Travailleur.se communautaire et au parapublic,2024-04-01 00:00:00,manque de logement et logement trop cher,Manque de logements et logements trop chers.,7- Pauvreté et apprauvrissement,7C- Manque de logements abordables
1,MRC de la Nouvelle-Beauce,Travailleur.se communautaire et au parapublic,2024-04-01 00:00:00,Manque de soins psychologique (immigration),Manque de soins psychologiques (immigration).,12- Défis liés à l'intégration des personnes i...,10G- Défis liés à l'intégration des personnes ...
2,MRC de la Nouvelle-Beauce,Travailleur.se communautaire et au parapublic,2024-04-01 00:00:00,Revenu insufisant pour combler les besoins de ...,Revenu insuffisant pour combler les besoins de...,7- Pauvreté et apprauvrissement,7B- Pauvreté (incluant insécurité financière)
3,Ville de Lévis,"Gestionnaire communautaire, parapublic ou du r...",2024-04-01 00:00:00,détresse psychologique(capacité d'adaptation ),Détresse psychologique (capacité d'adaptation),3- Bien-être et santé mentale,3A- Bien-être et santé mentale – Population gé...
4,Ville de Lévis,Professionnel.le du réseau de la santé et des ...,2024-04-01 00:00:00,"pauvreté, faible revenu, inquiétudes au niveau...","Pauvreté, faible revenu, inquiétudes au niveau...",7- Pauvreté et apprauvrissement,7B- Pauvreté (incluant insécurité financière)


In [10]:
# небольшое преобразование
data['Months_from_10_2021'] = data['Date_vigie_ok'].apply(months)
data['Num_categorie_nv1'] = data['Categorie_preoccup_nv1_ok'].apply(preoc_class)
data['Num_categorie_nv2'] = data['Categorie_preoccup_nv2'].apply(preoc2_class)
del data['Date_vigie_ok']
del data['Preoccup']

In [11]:
# построение списков первичной и вторичной системы классов
ncats1 = data['Num_categorie_nv1'].unique()
ncats2 = data['Num_categorie_nv2'].unique()

In [24]:
# обучение на всем датасете
train_dict = {} # {k: {noun: {'-': N_, verb: N_verb}}}
for i, row in data.iterrows():
    if not i%50:
        print(i)
    row_pairs = pairs(row['Preoccup_ok'])
    k = row['Num_categorie_nv1']
    train_dict = involve(row_pairs, k, train_dict)
    c = row['Num_categorie_nv2']
    train_dict = involve(row_pairs, c, train_dict)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500


In [25]:
# вычисление правильности на всем датасете (вывод по мере вычисления)
acc = 0
for i, row in data.iterrows():
    predicted = classify(row['Preoccup_ok'], train_dict, ncats1)
    real = row['Num_categorie_nv1']
    if predicted == real:
        acc+=1
    N=i+1
    if not i%50:
        print('шаг ', i, "точность ", acc/N)

шаг  0 точность  1.0
шаг  50 точность  0.5882352941176471
шаг  100 точность  0.6534653465346535
шаг  150 точность  0.6821192052980133
шаг  200 точность  0.6616915422885572
шаг  250 точность  0.6533864541832669
шаг  300 точность  0.6411960132890365
шаг  350 точность  0.6324786324786325
шаг  400 точность  0.6433915211970075
шаг  450 точность  0.6541019955654102
шаг  500 точность  0.6606786427145709
шаг  550 точность  0.662431941923775
шаг  600 точность  0.6655574043261231
шаг  650 точность  0.6620583717357911
шаг  700 точность  0.6676176890156919
шаг  750 точность  0.6644474034620506
шаг  800 точность  0.651685393258427
шаг  850 точность  0.6509988249118684
шаг  900 точность  0.6548279689234184
шаг  950 точность  0.6487907465825447
шаг  1000 точность  0.6503496503496503
шаг  1050 точность  0.6546146527117032
шаг  1100 точность  0.6530426884650318
шаг  1150 точность  0.6516072980017377
шаг  1200 точность  0.6544546211490425
шаг  1250 точность  0.6522781774580336
шаг  1300 точность  0.6487